In [1]:
from datafusion import SessionContext

ctx = SessionContext()
ctx.register_csv(
    "Bid", "./bid_data.csv"
)



In [2]:
import datafusion
print(datafusion.__version__)

40.1.0


In [3]:
from datafusion import substrait as dfs

# set environment variable RUST_BACKTRACE=1
# to see the backtrace of the panic

import os
os.environ['RUST_BACKTRACE'] = '1'

substrait_bytes = None
with open("window_avg_price.proto", "rb") as f:
    substrait_bytes = f.read()

# Deserialize the .proto file
substrait_plan = dfs.serde.deserialize_bytes(substrait_bytes)

# Get the query plan
df_logical_plan = dfs.consumer.from_substrait_plan(ctx, substrait_plan)

print(df_logical_plan)

# Execute it!
# results = ctx.create_dataframe_from_logical_plan(df_logical_plan)
# print(results)

Projection: Bid.auction, Bid.bidder, Bid.price, Bid.datetime, avg(Bid.price) PARTITION BY [Bid.auction] ORDER BY [Bid.datetime ASC NULLS FIRST] RANGE BETWEEN 5 PRECEDING AND 5 FOLLOWING AS avg_price
  WindowAggr: windowExpr=[[avg(Bid.price) PARTITION BY [Bid.auction] ORDER BY [Bid.datetime ASC NULLS FIRST] RANGE BETWEEN 5 PRECEDING AND 5 FOLLOWING]]
    TableScan: Bid projection=[auction, bidder, price, datetime]
